In [1]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._ 
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.split

Intitializing Scala interpreter ...

Spark Web UI available at http://mbpdejebaptiste:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1618495077210)
SparkSession available as 'spark'


import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.split


## Load des données

In [2]:
val karate_text = sc.textFile("data/soc-karate/soc-karate.mtx")


karate_text: org.apache.spark.rdd.RDD[String] = data/soc-karate/soc-karate.mtx MapPartitionsRDD[1] at textFile at <console>:38


## Preprocess

In [3]:
var df_karate_text = karate_text.toDF.withColumn("id",monotonicallyIncreasingId)


df_karate_text = df_karate_text.withColumn("rank", row_number().over(Window.orderBy("id")))
df_karate_text = df_karate_text.filter(df_karate_text("rank")>24)
df_karate_text = df_karate_text.drop("id","rank")

val df_karate_splitted = df_karate_text.withColumn("_tmp", split($"value", "\\ ")).select(
  $"_tmp".getItem(0).as("srcId"),
  $"_tmp".getItem(1).as("dstID"),
)

val rows: RDD[Row] = df_karate_splitted.rdd

var karate_edges = rows.map{ case Row(src:String, dist : String) => Edge(src.toLong, dist.toLong,1)}


df_karate_text: org.apache.spark.sql.DataFrame = [value: string]
df_karate_text: org.apache.spark.sql.DataFrame = [value: string]
df_karate_text: org.apache.spark.sql.DataFrame = [value: string]
df_karate_text: org.apache.spark.sql.DataFrame = [value: string]
df_karate_splitted: org.apache.spark.sql.DataFrame = [srcId: string, dstID: string]
rows: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[11] at rdd at <console>:51
karate_edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Int]] = MapPartitionsRDD[12] at map at <console>:53


## Creation du graphe

In [4]:
var default_user=(0, 0)
var graph = Graph.fromEdges(karate_edges, default_user)

default_user: (Int, Int) = (0,0)
graph: org.apache.spark.graphx.Graph[(Int, Int),Int] = org.apache.spark.graphx.impl.GraphImpl@4a4e91eb


In [5]:
graph.edges.collect()

res0: Array[org.apache.spark.graphx.Edge[Int]] = Array(Edge(2,1,1), Edge(3,1,1), Edge(3,2,1), Edge(4,1,1), Edge(4,2,1), Edge(4,3,1), Edge(5,1,1), Edge(6,1,1), Edge(7,1,1), Edge(7,5,1), Edge(7,6,1), Edge(8,1,1), Edge(8,2,1), Edge(8,3,1), Edge(8,4,1), Edge(9,1,1), Edge(9,3,1), Edge(10,3,1), Edge(11,1,1), Edge(11,5,1), Edge(11,6,1), Edge(12,1,1), Edge(13,1,1), Edge(13,4,1), Edge(14,1,1), Edge(14,2,1), Edge(14,3,1), Edge(14,4,1), Edge(17,6,1), Edge(17,7,1), Edge(18,1,1), Edge(18,2,1), Edge(20,1,1), Edge(20,2,1), Edge(22,1,1), Edge(22,2,1), Edge(26,24,1), Edge(26,25,1), Edge(28,3,1), Edge(28,24,1), Edge(28,25,1), Edge(29,3,1), Edge(30,24,1), Edge(30,27,1), Edge(31,2,1), Edge(31,9,1), Edge(32,1,1), Edge(32,25,1), Edge(32,26,1), Edge(32,29,1), Edge(33,3,1), Edge(33,9,1), Edge(33,15,1), Edge(33...


In [6]:
graph.vertices.collect()

res1: Array[(org.apache.spark.graphx.VertexId, (Int, Int))] = Array((20,(0,0)), (13,(0,0)), (19,(0,0)), (34,(0,0)), (15,(0,0)), (4,(0,0)), (21,(0,0)), (16,(0,0)), (22,(0,0)), (25,(0,0)), (28,(0,0)), (29,(0,0)), (11,(0,0)), (14,(0,0)), (32,(0,0)), (30,(0,0)), (24,(0,0)), (27,(0,0)), (33,(0,0)), (23,(0,0)), (1,(0,0)), (6,(0,0)), (17,(0,0)), (3,(0,0)), (7,(0,0)), (9,(0,0)), (8,(0,0)), (12,(0,0)), (18,(0,0)), (31,(0,0)), (26,(0,0)), (10,(0,0)), (5,(0,0)), (2,(0,0)))


### Calcul des degrés des noeuds

In [7]:
val newgraph = graph.outerJoinVertices(graph.degrees)((index,_,deg) => (index,deg.get))

newgraph: org.apache.spark.graphx.Graph[(org.apache.spark.graphx.VertexId, Int),Int] = org.apache.spark.graphx.impl.GraphImpl@145473c9


In [8]:
newgraph.triplets.collect()

res2: Array[org.apache.spark.graphx.EdgeTriplet[(org.apache.spark.graphx.VertexId, Int),Int]] = Array(((2,(2,9)),(1,(1,16)),1), ((3,(3,10)),(1,(1,16)),1), ((3,(3,10)),(2,(2,9)),1), ((4,(4,6)),(1,(1,16)),1), ((4,(4,6)),(2,(2,9)),1), ((4,(4,6)),(3,(3,10)),1), ((5,(5,3)),(1,(1,16)),1), ((6,(6,4)),(1,(1,16)),1), ((7,(7,4)),(1,(1,16)),1), ((7,(7,4)),(5,(5,3)),1), ((7,(7,4)),(6,(6,4)),1), ((8,(8,4)),(1,(1,16)),1), ((8,(8,4)),(2,(2,9)),1), ((8,(8,4)),(3,(3,10)),1), ((8,(8,4)),(4,(4,6)),1), ((9,(9,5)),(1,(1,16)),1), ((9,(9,5)),(3,(3,10)),1), ((10,(10,2)),(3,(3,10)),1), ((11,(11,3)),(1,(1,16)),1), ((11,(11,3)),(5,(5,3)),1), ((11,(11,3)),(6,(6,4)),1), ((12,(12,1)),(1,(1,16)),1), ((13,(13,2)),(1,(1,16)),1), ((13,(13,2)),(4,(4,6)),1), ((14,(14,5)),(1,(1,16)),1), ((14,(14,5)),(2,(2,9)),1), ((14,(14,...


In [9]:
newgraph.vertices.collect()

res3: Array[(org.apache.spark.graphx.VertexId, (org.apache.spark.graphx.VertexId, Int))] = Array((20,(20,3)), (13,(13,2)), (19,(19,2)), (34,(34,17)), (15,(15,2)), (4,(4,6)), (21,(21,2)), (16,(16,2)), (22,(22,2)), (25,(25,3)), (28,(28,4)), (29,(29,3)), (11,(11,3)), (14,(14,5)), (32,(32,6)), (30,(30,4)), (24,(24,5)), (27,(27,2)), (33,(33,12)), (23,(23,2)), (1,(1,16)), (6,(6,4)), (17,(17,2)), (3,(3,10)), (7,(7,4)), (9,(9,5)), (8,(8,4)), (12,(12,1)), (18,(18,2)), (31,(31,4)), (26,(26,3)), (10,(10,2)), (5,(5,3)), (2,(2,9)))


ETAPE 1: Variables utiles pour calculer le gain en modularité lors de la suppression d'un noeud d'une communauté

In [10]:
def sigmaIn1(i:Long, comId: Long, graph : Graph[(Long, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcAttr._1==comId && triplet.dstAttr._1==comId) &&
                                                    !(triplet.srcId==i || triplet.dstId==i)))
    return graphVar.count.toDouble

}


def sigmaTot1(i:Long, comId: Long, graph : Graph[(Long, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcAttr._1==comId && triplet.dstAttr._1!=comId)||(triplet.srcAttr._1!=comId && triplet.dstAttr._1==comId) && !(triplet.srcId==i || triplet.dstId==i)))
    return graphVar.count.toDouble
}

sigmaIn1: (i: Long, comId: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double
sigmaTot1: (i: Long, comId: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double


ETAPE 2: Variables utiles pour calculer le gain en modularité lors de l'ajout d'un noeud à une communauté

In [11]:
def sigmaIn2(comId: Long, graph : Graph[(Long, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => triplet.srcAttr._1==comId && triplet.dstAttr._1==comId))
    return graphVar.count.toDouble
}

def kiin2(i:Long, comId : Long, graph : Graph[(Long, Int),Int]) : Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcAttr._1==i && triplet.dstAttr._1==comId) ||(triplet.srcAttr._1==comId && triplet.dstAttr._1==i)))
    return graphVar.count.toDouble
}

def sigmaTot2(comId: Long, graph : Graph[(Long, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcAttr._1==comId && triplet.dstAttr._1!=comId)||(triplet.srcAttr._1!=comId && triplet.dstAttr._1==comId)))
    return graphVar.count.toDouble
}

sigmaIn2: (comId: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double
kiin2: (i: Long, comId: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double
sigmaTot2: (comId: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double


Variables utiles au 2 étapes

In [12]:
def ki(i: Long, graph : Graph[(Long, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => triplet.srcId==i || triplet.dstId==i))
    return graphVar.count.toDouble
}

def kiin(i:Long, comId : Long, graph : Graph[(Long, Int),Int]) : Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcAttr._1==i && triplet.dstAttr._1==comId) ||(triplet.srcAttr._1==comId && triplet.dstAttr._1==i)))
    return graphVar.count.toDouble
}

ki: (i: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double
kiin: (i: Long, comId: Long, graph: org.apache.spark.graphx.Graph[(Long, Int),Int])Double


Calcul du différentiel de modularité

In [13]:
def dq(sigmain:Double, sigmatot:Double, kivar:Double, kiinvar:Double, m:Long):Double = {
    return (((sigmain+kiinvar)/(2*m) - scala.math.pow((sigmatot+kivar)/(2*m),2)) - (sigmain/(2*m) - scala.math.pow(sigmatot/(2*m),2) - scala.math.pow(kivar/(2*m),2)))
}

dq: (sigmain: Double, sigmatot: Double, kivar: Double, kiinvar: Double, m: Long)Double


In [14]:
def maxCom(setDq : Set[(Int, Double)]) : (Int, Double) = {
    var max = -1000.0
    var ind = -1
    setDq.foreach(x => {
        if (x._2>max)
        {
            ind=x._1
            max=x._2
        }
    })
    return (ind,max)
}

maxCom: (setDq: Set[(Int, Double)])(Int, Double)


In [17]:
def moveItoC(i:Int, comId : Int, graph:Graph[(VertexId, Int),Int]) : Graph[(VertexId, Int),Int] = {
    val newVertices = graph.mapVertices { case (id, attr) => if (id==i) (comId.toLong, attr._2) else attr }
    return newVertices
}

moveItoC: (i: Int, comId: Int, graph: org.apache.spark.graphx.Graph[(org.apache.spark.graphx.VertexId, Int),Int])org.apache.spark.graphx.Graph[(org.apache.spark.graphx.VertexId, Int),Int]


In [21]:
def communities(graph:Graph[(Long,Int),Int]) : Graph[(Long,Int),Int] = {
    var currentGraph = graph
    var nbVertices = graph.vertices.count
    var m = graph.edges.count

    var i = 0
    var keepGoing = true
    var j=0
    for (j <- 1 to 2){
        println(j)
        keepGoing=false
        println(((currentGraph.vertices.collect { case t => t._2._1 }).collect.distinct).size)
        for (i<- 1 to nbVertices.toInt){
            println("Noeud considéré : "+i+ '\n')
            /*get the community and ki */
            var vertice = (currentGraph.vertices.collect{ case t if t._1 == i => t._2}).collect
            var comId = vertice(0)._1.toInt
            println("Communauté de "+i+" : "+comId)
            var kivar = vertice(0)._2

            /* calculate dq1 */
            var kiin1 = kiin(i, comId, currentGraph)
            var simgain1 = sigmaIn1(i, comId, currentGraph)
            var simgatot1 = sigmaTot1(i, comId, currentGraph)
            var dq1 = dq(sigmain1, sigmatot1, kivar, kiin1, m)

            /*look all combis : we need to know how many communities are left and which one*/
            var communities = (currentGraph.vertices.collect { case t => t._2._1 }).collect.distinct

            var setDq =Set(): Set[(Int,Double)]
            communities.foreach( a => {
                if (a!=comId){
                    var sigmain2 = sigmaIn2(a, newgraph)
                    var sigmatot2 = sigmaTot2(a, newgraph)
                    var kiin2 = kiin(i,a, newgraph)
                    var dq2 = dq(sigmain2, sigmatot2, kivar, kiin2, m)
                    setDq+=((a.toInt,dq2))
                }
            })

            /* get best combi and move if larger than 0 */
            var bestCombi = maxCom(setDq)
            if (bestCombi._2 > 0 ) {
                
                var communities = (currentGraph.vertices.collect { case t => t._2._1 }).collect.distinct
                println("On met "+ i +" dans "+bestCombi._1+ '\n')
                var updatedGraph = moveItoC(i, bestCombi._1, currentGraph)
                currentGraph=updatedGraph

                /*Un changement est fait*/
                keepGoing=true

            }
            
    }
    println(((currentGraph.vertices.collect { case t => t._2._1 }).collect.distinct).size)
    }
    return currentGraph
}

communities: (graph: org.apache.spark.graphx.Graph[(Long, Int),Int])org.apache.spark.graphx.Graph[(Long, Int),Int]


### TEST

On veut savoir s'il est intéressant de placer le noeud 1 dans la communauté 14

In [19]:
var sigmain1 = sigmaIn1(1,14,newgraph)
var sigmatot1 = sigmaTot1(1,14,newgraph)
var kivar = ki(1, newgraph)
var kiinvar = kiin2(1,14,newgraph)
var m=newgraph.edges.count()/2

sigmain1: Double = 0.0
sigmatot1: Double = 5.0
kivar: Double = 16.0
kiinvar: Double = 1.0
m: Long = 39


In [146]:
var dq1 = DQ(sigmain1, sigmatot1, kivar, kiinvar, m , newgraph)

dq1: Double = -0.013477975016436564


In [ ]:
var a =communities(newgraph)

1
34
Noeud considéré : 1

Communauté de 1 : 1


In [ ]:
(a.vertices.collect{ case t => t._2._1 }).collect.distinct